In [1]:
 import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix, precision_score, f1_score, recall_score

## Data Exploration

Dataset Description:

Transaction_ID: Unique identifier for each transaction.

Timestamp: Date and time of the transaction.

Vehicle_Type: Type of vehicle involved in the transaction.

FastagID: Unique identifier for Fastag.

TollBoothID: Identifier for the toll booth.

Lane_Type: Type of lane used for the transaction.

Vehicle_Dimensions: Dimensions of the vehicle.

Transaction_Amount: Amount associated with the transaction.

Amount_paid: Amount paid for the transaction.

Geographical_Location: Location details of the transaction.

Vehicle_Speed: Speed of the vehicle during the transaction.

Vehicle_Plate_Number: License plate number of the vehicle.

Fraud_indicator: Binary indicator of fraudulent activity (target variable)

In [3]:
data = pd.read_csv('FastagFraudDetection.csv')
data.head()

,Transaction_ID,Timestamp,Vehicle_Type,FastagID,TollBoothID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Geographical_Location,Vehicle_Speed,Vehicle_Plate_Number,Fraud_indicator
0,1,1/6/2023 11:20,Bus,FTG-001-ABC-121,A-101,Express,Large,350,120,"13.059816123454882, 77.77068662374292",65,KA11AB1234,Fraud
1,2,1/7/2023 14:55,Car,FTG-002-XYZ-451,B-102,Regular,Small,120,100,"13.059816123454882, 77.77068662374292",78,KA66CD5678,Fraud
2,3,1/8/2023 18:25,Motorcycle,NaN,D-104,Regular,Small,0,0,"13.059816123454882, 77.77068662374292",53,KA88EF9012,Not Fraud
3,4,1/9/2023 2:05,Truck,FTG-044-LMN-322,C-103,Regular,Large,350,120,"13.059816123454882, 77.77068662374292",92,KA11GH3456,Fraud
4,5,1/10/2023 6:35,Van,FTG-505-DEF-652,B-102,Express,Medium,140,100,"13.059816123454882, 77.77068662374292",60,KA44IJ6789,Fraud


In [4]:
data.shape

(5000, 13)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Transaction_ID         5000 non-null   int64 
 1   Timestamp              5000 non-null   object
 2   Vehicle_Type           5000 non-null   object
 3   FastagID               4451 non-null   object
 4   TollBoothID            5000 non-null   object
 5   Lane_Type              5000 non-null   object
 6   Vehicle_Dimensions     5000 non-null   object
 7   Transaction_Amount     5000 non-null   int64 
 8   Amount_paid            5000 non-null   int64 
 9   Geographical_Location  5000 non-null   object
 10  Vehicle_Speed          5000 non-null   int64 
 11  Vehicle_Plate_Number   5000 non-null   object
 12  Fraud_indicator        5000 non-null   object
dtypes: int64(4), object(9)
memory usage: 507.9+ KB


## Feature Engineering

Identify and engineer relevant features that contribute to fraud detection accuracy.

### OneHot Encoding

In [6]:
data['Vehicle_Type'].unique()

array(['Bus ', 'Car', 'Motorcycle', 'Truck', 'Van', 'Sedan', 'SUV'],
      dtype=object)

In [7]:
Lane_order=['Express', 'Regular']
Vehicle_Dimensions_order=['Large', 'Small', 'Medium']
Fraud_indicator_order=['Not Fraud','Fraud']

In [8]:
ohe = OneHotEncoder()
encode0 = ohe.fit_transform(data[['Vehicle_Type']]).toarray()

In [9]:
feature_labels = ohe.categories_
np.array(feature_labels).ravel()

array(['Bus ', 'Car', 'Motorcycle', 'SUV', 'Sedan', 'Truck', 'Van'],
      dtype=object)

In [10]:
feature_labels = np.array(feature_labels).ravel()
print(feature_labels)

['Bus ' 'Car' 'Motorcycle' 'SUV' 'Sedan' 'Truck' 'Van']


In [11]:
features = pd.DataFrame(encode0, columns = feature_labels)

In [12]:
df_new = pd.concat([data, features], axis=1)

In [13]:
new_dataset=df_new.drop(['Timestamp','FastagID','Vehicle_Type','TollBoothID','Geographical_Location','Vehicle_Plate_Number'], axis=1)

In [14]:
new_dataset

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Fraud_indicator,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
0,1,Express,Large,350,120,65,Fraud,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Regular,Small,120,100,78,Fraud,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,Regular,Small,0,0,53,Not Fraud,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Regular,Large,350,120,92,Fraud,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,Express,Medium,140,100,60,Fraud,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,Regular,Large,330,330,81,Not Fraud,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4996,4997,Express,Medium,125,125,64,Not Fraud,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4997,4998,Regular,Medium,115,115,93,Not Fraud,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4998,4999,Express,Large,145,145,57,Not Fraud,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Ordinal Encoding

In [15]:
encode1 = OrdinalEncoder(categories=[Lane_order])
encode2 = OrdinalEncoder(categories=[Vehicle_Dimensions_order])
encode3 = OrdinalEncoder(categories=[Fraud_indicator_order])

In [16]:
encode1.fit(new_dataset[['Lane_Type']])
encode2.fit(new_dataset[['Vehicle_Dimensions']])
encode3.fit(new_dataset[['Fraud_indicator']])

OrdinalEncoder(categories=[['Not Fraud', 'Fraud']])

In [17]:
new_lane=pd.DataFrame(encode1.transform(new_dataset[['Lane_Type']]))
new_dimensuions=pd.DataFrame(encode2.transform(new_dataset[['Vehicle_Dimensions']]))
new_fraud_indicator=pd.DataFrame(encode3.transform(new_dataset[['Fraud_indicator']]))

In [18]:
new_dataset['Lane_Type']= new_lane
new_dataset['Vehicle_Dimensions']= new_dimensuions
new_dataset['Fraud_indicator']=new_fraud_indicator

In [19]:
# dataset information
new_dataset

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Fraud_indicator,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
0,1,0.0,0.0,350,120,65,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,1.0,120,100,78,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,1.0,0,0,53,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,1.0,0.0,350,120,92,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,0.0,2.0,140,100,60,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,1.0,0.0,330,330,81,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4996,4997,0.0,2.0,125,125,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4997,4998,1.0,2.0,115,115,93,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4998,4999,0.0,0.0,145,145,57,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
## Handlling missing values

# handlling missing values in dataset with dropping method
data = new_dataset.dropna(how='any')

In [21]:
# checking the missing values of each column
data.isnull().sum()

Transaction_ID        0
Lane_Type             0
Vehicle_Dimensions    0
Transaction_Amount    0
Amount_paid           0
Vehicle_Speed         0
Fraud_indicator       0
Bus                   0
Car                   0
Motorcycle            0
SUV                   0
Sedan                 0
Truck                 0
Van                   0
dtype: int64

In [22]:
#dristibution of legit and fraud transactions
data['Fraud_indicator'].value_counts()

Fraud_indicator
0.0    4017
1.0     983
Name: count, dtype: int64

### Highly Unblanced dataset

### 0-> normal transaction

### 1-> Fraud transaction

In [25]:
# separating data for analysis
legit = data[data.Fraud_indicator == 0]
fraud = data[data.Fraud_indicator == 1]

In [26]:
print(legit.shape)
print(fraud.shape)

(4017, 14)
(983, 14)


In [27]:
#statistical method of the data
legit.Transaction_Amount.describe()

count    4017.000000
mean      153.110530
std       114.435986
min         0.000000
25%        90.000000
50%       125.000000
75%       290.000000
max       350.000000
Name: Transaction_Amount, dtype: float64

In [28]:
fraud.Transaction_Amount.describe()

count    983.000000
mean     193.555443
std       97.465586
min       60.000000
25%      120.000000
50%      145.000000
75%      300.000000
max      350.000000
Name: Transaction_Amount, dtype: float64

In [29]:
#compare the values for both transactions
data.groupby('Fraud_indicator').mean()

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
Fraud_indicator,,,,,,,,,,,,,
0.0,2618.246950,0.588748,0.865820,153.110530,153.11053,67.731392,0.134180,0.147374,0.177745,0.131939,0.137665,0.138412,0.132686
1.0,2019.330621,0.501526,0.819939,193.555443,92.83825,68.340793,0.180061,0.124110,0.000000,0.187182,0.163784,0.160732,0.184130


### Under Sampling

In [30]:
legit_sample = legit.sample(n=983)

### Concatenating two DataFrames

In [31]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [32]:
new_dataset.head()

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Fraud_indicator,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
2003,2004,1.0,1.0,90,90,90,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1924,1925,0.0,0.0,140,140,58,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3091,3092,1.0,0.0,290,290,49,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3352,3353,0.0,2.0,100,100,45,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
109,110,1.0,2.0,140,140,87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [33]:
#dristibution of legit and fraud transactions in new dataset
new_dataset['Fraud_indicator'].value_counts()

Fraud_indicator
0.0    983
1.0    983
Name: count, dtype: int64

In [35]:
new_dataset.groupby('Fraud_indicator').mean()

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
Fraud_indicator,,,,,,,,,,,,,
0.0,2604.504578,0.571719,0.907426,149.399797,149.399797,68.107833,0.115972,0.165819,0.163784,0.136317,0.136317,0.129196,0.152594
1.0,2019.330621,0.501526,0.819939,193.555443,92.838250,68.340793,0.180061,0.124110,0.000000,0.187182,0.163784,0.160732,0.184130


## Splitting the data into features and targets

In [37]:
X = new_dataset.drop(columns='Fraud_indicator', axis=1)
Y = new_dataset['Fraud_indicator']

## Split data into training and testing

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8, stratify=Y, random_state=2)

In [39]:
print(X.shape, X_train.shape, X_test.shape)

(1966, 13) (1572, 13) (394, 13)


In [40]:
pd.DataFrame(X_train)

,Transaction_ID,Lane_Type,Vehicle_Dimensions,Transaction_Amount,Amount_paid,Vehicle_Speed,Bus,Car,Motorcycle,SUV,Sedan,Truck,Van
2460,2461,0.0,0.0,330,330,67,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3758,3759,0.0,2.0,120,100,57,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2181,2182,1.0,2.0,125,90,95,0.0,0.0,0.0,0.0,0.0,0.0,1.0
62,63,0.0,0.0,180,180,67,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4693,4694,1.0,1.0,0,0,84,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,272,1.0,2.0,160,160,87,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3366,3367,0.0,2.0,100,100,45,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1312,1313,0.0,0.0,340,340,66,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3674,3675,0.0,2.0,120,120,62,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [41]:
pd.DataFrame(Y_train)

,Fraud_indicator
2460,0.0
3758,1.0
2181,1.0
62,0.0
4693,0.0
...,...
271,0.0
3366,0.0
1312,0.0
3674,0.0


## Model Development

In [43]:
Models = {
    "Decision Tree":DecisionTreeClassifier(),
    "Random Forest":RandomForestClassifier(),
    "Logistic Regression":LogisticRegression(),
    "SVM Classification": SVC()
}

for i in range (len(list(Models))):
    Model=list(Models.values())[i]

    #train Model
    Model.fit(X_train, Y_train)

    #Make predictions
    Y_train_pred = Model.predict(X_train)
    Y_test_pred = Model.predict(X_test)

    #Training Performance
    model_train_Accuracy = accuracy_score(Y_train, Y_train_pred)
    model_train_Precision = precision_score(Y_train, Y_train_pred)
    model_train_recall = recall_score(Y_train, Y_train_pred)
    model_train_F1 = f1_score(Y_train, Y_train_pred, average='weighted')

    #Testing Performance
    model_test_Accuracy = accuracy_score(Y_test, Y_test_pred)
    model_test_Precision = precision_score(Y_test, Y_test_pred)
    model_test_recall = recall_score(Y_test, Y_test_pred)
    model_test_F1 = f1_score(Y_test, Y_test_pred, average='weighted')

    print(list(Models.keys())[i])

    print("Models Performance for Training Set")
    print("- Accuracy: {:.4f}".format(model_train_Accuracy))
    print("- Precision: {:.4f}".format(model_train_Precision))
    print("- Recall: {:.4f}".format(model_train_recall))
    print("- F1 Score: {:.4f}".format(model_train_F1))


    print("--------------------------")

    print("Models Performance for Testing Set")
    print("- Accuracy: {:.4f}".format(model_test_Accuracy))
    print("- Precision: {:.4f}".format(model_test_Precision))
    print("- Recall: {:.4f}".format(model_test_recall))
    print("- F1 Score: {:.4f}".format(model_test_F1))

    print('='*35)
    print('\n')

Decision Tree
Models Performance for Training Set
- Accuracy: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- F1 Score: 1.0000
--------------------------
Models Performance for Testing Set
- Accuracy: 0.9924
- Precision: 1.0000
- Recall: 0.9848
- F1 Score: 0.9924


Random Forest
Models Performance for Training Set
- Accuracy: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- F1 Score: 1.0000
--------------------------
Models Performance for Testing Set
- Accuracy: 0.9797
- Precision: 0.9948
- Recall: 0.9645
- F1 Score: 0.9797


Logistic Regression
Models Performance for Training Set
- Accuracy: 0.9771
- Precision: 1.0000
- Recall: 0.9542
- F1 Score: 0.9771
--------------------------
Models Performance for Testing Set
- Accuracy: 0.9822
- Precision: 1.0000
- Recall: 0.9645
- F1 Score: 0.9822




C:\Users\priya\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM Classification
Models Performance for Training Set
- Accuracy: 0.6838
- Precision: 0.7213
- Recall: 0.5992
- F1 Score: 0.6816
--------------------------
Models Performance for Testing Set
- Accuracy: 0.7259
- Precision: 0.7947
- Recall: 0.6091
- F1 Score: 0.7221


